Name: Allyson Busch

Date: 12/07/2019

# Paranormal Romance Title Generator

The Kaggle data set that the data set originally came from can be found at https://www.kaggle.com/rtatman/paranormal-romance-novel-titles and contains a list of 4,000 paranormal romance novel titles that were scraped from the web by Mark Riedl. 

Paranormal Romance is a subgenre of the Romance genre of books, and frequently combines fantasy and romance elements. Notable examples of this genre in popular fiction is the Twilight series as well as The Southern Vampire Mysteries, better known as True Blood in it's television adaptation.

The goal of this project is to create a Markov chain text generator.

In [1]:
# importing the libraries
import numpy as np
import pandas as pd
import random

In [8]:
# importing the dataset

pararomance = pd.read_csv('pararomance.csv', header = None, usecols = [0])

In [9]:
pararomance

,0
0,Moonshadow
1,Angels' Blood (Guild Hunter...
2,The Seven Series Boxed Set...
3,The Darkest Promise: A Dark
4,The Summit (Second Paranormal...
5,The Skeleton in the Closet
6,Shadow Reaper
7,Dark Lover (Black Dagger...
8,The Chosen (Signed Book)...
9,Silence Fallen (Mercy Thompson...


The imported data set contains 4000 titles, as expected.

## Exploratory Data Analysis

In [10]:
pararomance.shape

(4000, 1)

In [11]:
pararomance.describe()

,0
count,4000
unique,3918
top,Paranormal Shifter Romance...
freq,7


We can tell from the table above that there are duplicate titles (perhaps a series?), so for this exercise I would like to delete duplicates.

In [12]:
pararomance.drop_duplicates(keep = 'first')

,0
0,Moonshadow
1,Angels' Blood (Guild Hunter...
2,The Seven Series Boxed Set...
3,The Darkest Promise: A Dark
4,The Summit (Second Paranormal...
5,The Skeleton in the Closet
6,Shadow Reaper
7,Dark Lover (Black Dagger...
8,The Chosen (Signed Book)...
9,Silence Fallen (Mercy Thompson...


I now want to rename the column to Title, just for ease of creating the model.

In [20]:
pararomance.rename(columns = {0: "Title"}, inplace=True)
pararomance.head()

,Title
0,Moonshadow
1,Angels' Blood (Guild Hunter...
2,The Seven Series Boxed Set...
3,The Darkest Promise: A Dark
4,The Summit (Second Paranormal...


In [21]:
pararomance.isnull().sum()

Title    0
dtype: int64

There are no null values in the dataset, so we can move on to removing the elipses and \n that are included in the titles.

In [24]:
pararomance_updated = [i for i in pararomance.Title.str.strip(' ') if i != '']
pararomance_updated = [word.replace(".", "") for word in pararomance_updated]
data = []
for i in range(0, len(pararomance_updated)):
    data.append(pararomance_updated[i].rstrip('\n'))

In [26]:
print(data[0:5])

['Moonshadow', "Angels' Blood (Guild Hunter", 'The Seven Series Boxed Set', 'The Darkest Promise: A Dark', 'The Summit (Second Paranormal']


## Markov Chain

First I want to create the empty list for the Markov chain and then link the chain using key values.

In [27]:
markov = {i:[] for i in data}

for before, after in zip(data, data[1:]):
    markov[before].append(after)

Now we are randomly selecting a value.

In [28]:
new = list(markov.keys())
seed = random.randrange(len(new))
currentWord = random.choice(new)
sentence = [currentWord]

for i in range(0, random.randrange(1,2)):
    check = markov[currentWord]
    if (len(check) > 0):
        nextWord = random.choice(check)
        sentence.append(nextWord)
        currentWord = nextWord
    else:
        currentWord = random.choice(new)
        
print(" ".join(sentence))

Bitten Eternal Vows


In [29]:
new = list(markov.keys())
seed = random.randrange(len(new))
currentWord = random.choice(new)
sentence = [currentWord]

for i in range(0, random.randrange(1,2)):
    check = markov[currentWord]
    if (len(check) > 0):
        nextWord = random.choice(check)
        sentence.append(nextWord)
        currentWord = nextWord
    else:
        currentWord = random.choice(new)
        
print(" ".join(sentence))

Dark Secret (Dark Series #15) Kiss of Midnight (Midnight


In [30]:
new = list(markov.keys())
seed = random.randrange(len(new))
currentWord = random.choice(new)
sentence = [currentWord]

for i in range(0, random.randrange(1,2)):
    check = markov[currentWord]
    if (len(check) > 0):
        nextWord = random.choice(check)
        sentence.append(nextWord)
        currentWord = nextWord
    else:
        currentWord = random.choice(new)
        
print(" ".join(sentence))

In Chains Bound by Flame: A Novel of the


As you can see, this code can just be duplicated to create a new title. Some are better than others, as you can see from the examples above. What are the applications of this project? 

Often writers use title generators or character generators to jump start their writing. Generators like this can provide ideas for a writer who may be stuck, or who may want to experiment with short stories that could result from the title. Exercises for quick writing are also common from generators.